# Lightkurve Tutorial
Mit `lightkurve` lassen sich Daten von Kepler und TESS herunterladen und die Lichtkurve extrahieren. In diesem Tutorial gehen wir zuerst ein Beispiel, KELT-11 b durch. Im Anschluss extrahiert ihr die Lichtkurve für den Planeten, den ihr analysieren werdet.
Mehr Infos un Tutorials zu `lightkurve` könnt ihr hier finden: https://lightkurve.github.io/lightkurve/index.html

In [ ]:
from astropy import units as u
import lightkurve as lk
import matplotlib.pyplot as plt

## Target Pixel File
Zuerst laden wir uns die Imaging Data herunter. Das machen wir mit der `search_targetpixelfile` Funktion. Hier müssen wir den Namen unseres targets (in unserem Fall KELT-11 oder auch TIC 55092869), den Autor (für uns SPOC) und die Observationskadenz einfügen.

In [ ]:
# Searches for all available TESS data
search_result = lk.search_targetpixelfile("TIC 55092869")
search_result

In [ ]:
# Searches for Data processed by SPOC and with an exposure time of 120 seconds
search_result = lk.search_targetpixelfile("TIC 55092869",author='SPOC',exptime=120)
search_result

Als Beispiel analysieren wir nur einen Sektor. Hier nehmen wir Sektor 9.

In [ ]:
# Searches for Data processed by SPOC, with an exposure time of 120 seconds in Sector 9
search_result = lk.search_targetpixelfile("TIC 55092869",author='SPOC',exptime=120, sector=9)
search_result

Nun laden wir die Daten herunter.

In [ ]:
tpf = search_result.download()

Und plotten die Daten:

In [ ]:
tpf.plot(column='flux')
plt.show()

Die beste Aperture (die ausgewählten Pixel für folgende Analysen) ist auch in diesem File gespeichert. Diese können wir uns wie folgt anschauen:

In [ ]:
tpf.plot(aperture_mask=tpf[0].pipeline_mask);
plt.show()

## Konversion zur Lichtkurve
Das Targetpixelfile lässt sich zur Lichtkurve mit der `to_lichtcurve` Funktion konvertieren. Dabei wird der Flux innerhalb der Aperture für jeden aufgenommen Zeitpunkt gelistet.

In [ ]:
lc = tpf.to_lightcurve()
lc

Diese Lichtkurve können wir uns so anschauen:

In [ ]:
lc.plot()
plt.show()

## Analyse der Lichtkurven
Nun annalysieren wir alle vorhandenen Lichtkurven und schauen, ob wir selbst den Planeten finden können. Dazu laden wir zunächst alle verfügbaren Lichtkurven herunter. Diese schauen wir uns in einem Plot an.
KELT-11 wurde in 3 Sektoren observiert, die wir hier in den drei verschiedenen Farben sehen können.

In [ ]:
search_result = lk.search_lightcurve("TIC 55092869", author='SPOC', cadence=120)

# Download all available Kepler light curves
lc_collection = search_result.download_all()
lc_collection.plot()
plt.show()

Wir sehen, dass die Sektoren in Flux voneinander versetzt sind. Um sie auf die selbe Ebene zu bringen, normalisieren wir den Flux (`stitch`). Gleichzeitig entfernen wir Trends mit der `flatten` Funktion und entfernen Outlier (`remove_outliers`).
Die Lichtkurven sehen dann so aus:

In [ ]:
lc = lc_collection.stitch().flatten(window_length=901).remove_outliers()
lc.plot()
plt.show()

## Box Least Squares (BLS) Periodogram
Um periodische Signal zu finden nutzen wir BLS. Dies werden wir gemeinsam im Kurs durchgehen. Mehr Infos könnt ihr hier finden: https://docs.astropy.org/en/stable/timeseries/bls.html 

In [ ]:
import numpy as np
# Create array of periods to search
period = np.linspace(1, 20, 10000)
# Create a BLSPeriodogram
bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500)
bls.plot()
plt.show()

Wir sehen ein deutliches Signal knapp unter 5 Tagen. Wir können die Periode dieses Signals herausfinden, indem wir die Periode der maximales Power ausgeben:

In [ ]:
planet_b_period = bls.period_at_max_power
planet_b_t0 = bls.transit_time_at_max_power
planet_b_dur = bls.duration_at_max_power

# Check the value for period
planet_b_period

Dies ist ähnlich zu der bekannten Periode von KELT-11b.

Wir können die Lichtkurve nun in der Phase falten, sodass alle Transits übereinander liegen. Dadurch sehen wir den Transit klarer.

In [ ]:

ax = lc.fold(period=planet_b_period, epoch_time=planet_b_t0).scatter()
ax.set_xlim(-5, 5)
plt.show()

Wir können diesen Transit auch einmal mit dem BLS model zeigen:

In [ ]:
# Create a BLS model using the BLS parameters
planet_b_model = bls.get_transit_model(period=planet_b_period,
                                       transit_time=planet_b_t0,
                                       duration=planet_b_dur)

ax = lc.fold(planet_b_period, planet_b_t0).scatter()
planet_b_model.fold(planet_b_period, planet_b_t0).plot(ax=ax, c='r', lw=2)
ax.set_xlim(-5, 5)
plt.show()

## Gibt es noch weitere Planeten in den Daten?

Dazu entfernen wir den Transit des Planeten, den wir bereits kennen, indem wir eine Maske kreieren. 

In [ ]:
planet_b_mask = bls.get_transit_mask(period=planet_b_period,
                                     transit_time=planet_b_t0,
                                     duration=planet_b_dur)

Hier stellen wir die Lichtkurve ohne Transit in schwarz und die bekannten Transits on rot dar. 

In [ ]:
masked_lc = lc[~planet_b_mask]
ax = masked_lc.scatter()
lc[planet_b_mask].scatter(ax=ax, c='r', label='Masked')
plt.show()

Nun nutzen wir wieder ein BLS Periodogram, allerdings nur mit den Daten, die den bekannten Transit nicht enthalten:

In [ ]:
period = np.linspace(1, 30, 1000)
bls = masked_lc.to_periodogram('bls', period=period, frequency_factor=500)
bls.plot()
plt.show()

Wir sehen ein Signal knapp unter 10 Tagen. Wir können die Periode dieses Signals herausfinden, indem wir die Periode der maximales Power ausgeben:

In [ ]:
planet_c_period = bls.period_at_max_power
planet_c_t0 = bls.transit_time_at_max_power
planet_c_dur = bls.duration_at_max_power

# Check the value for period
planet_c_period

Auch hier können wir den Transit in Phase falten und das BLS model darstellen:

In [ ]:
planet_c_model = bls.get_transit_model(period=planet_c_period,
                                       transit_time=planet_c_t0,
                                       duration=planet_c_dur)

ax = lc.fold(planet_c_period, planet_c_t0).scatter()
planet_c_model.fold(planet_c_period, planet_c_t0).plot(ax=ax, c='r', lw=2)
ax.set_xlim(-5, 5)
plt.show()

Wir sehen, dass im Model nur ein Transit in der Mitte liegt, allerdings sehen wir auch Datenpunkte die einen geringenren Flux haben rechts und links vom Plot. Diese können wir besser zeigen, wenn wir den Flux einmal "binnen":

In [ ]:
ax = masked_lc.fold(planet_c_period, planet_c_t0).scatter()
masked_lc.fold(planet_c_period, planet_c_t0).bin(.1).scatter(ax=ax, c='r', lw=2,
                                                          label='Binned Flux')
ax.set_xlim(-5, 5)
plt.show()

Wir sehen hier deutlich klarer, dass der Transit auch am Rand von diesem Plot liegt. Das heißt der Transit passiert zu einer Period, die die Hälfte des Models ist. Das ist somit die Period, die wir zuvor gefunden haben und es handelt sich bei diesem Signal nicht um einen neuen Planeten.

Hast Du eine Idee, wieso der Algorithmus dieses Signal troztdem gefunden hat?